In [13]:
import numpy as np
import csv
from PIL import Image

# Open the temperature color map image
image = Image.open('../data/img/og/2020_LST.bmp')

# Convert the image to numpy array
image_array = np.array(image)

# Define the temperature range
min_temp = 10  # Minimum temperature in degrees Celsius
max_temp = 50  # Maximum temperature in degrees Celsius

# Convert RGB to temperature values
temperature_data = []
height, width, _ = image_array.shape
for y in range(height):
    for x in range(width):
        pixel = image_array[y, x]
         # Interpolate temperature value based on pixel color
        r, g, _ = pixel
        if [r,g] == [0,0]:
            temperature = -1
            temperature_data.append([x, y, temperature])  # Save spatial coordinates and temperature
        else:
            temperature = min_temp + (max_temp - min_temp) * (1 - g / 255)
            temperature_data.append([x, y, int(temperature)])  # Save spatial coordinates and temperature

# Save temperature data to CSV file
with open('temperature_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['X', 'Y', 'Temperature'])  # Header row
    writer.writerows(temperature_data)

print("Temperature data extracted and saved to temperature_data.csv")


Temperature data extracted and saved to temperature_data.csv


In [16]:
import numpy as np
import csv
from PIL import Image

# Read temperature data from CSV file
temperature_data = []
with open('./temperature_data.csv', 'r') as csvfile:
    reader = csv.reader(csvfile)
    next(reader)  # Skip header row
    for row in reader:
        x, y, temperature = map(int, row)
        temperature_data.append((x, y, temperature))

# Find min and max temperature values
temperatures = [temp for _, _, temp in temperature_data]
# find min temperaute but skip -1
temperat = [temp for temp in temperatures if temp != -1]
min_temp = min(temperat)
max_temp = max(temperatures)
print(min_temp, max_temp)
# Create an empty image array
width = max(x for x, _, _ in temperature_data) + 1
height = max(y for _, y, _ in temperature_data) + 1
image_array = np.zeros((height, width, 3), dtype=np.uint8)

# Convert temperature values back to color and fill the image array
for x, y, temperature in temperature_data:
    if temperature == -1:
        color = (0,0,0)
    else:
        # Interpolate color based on temperature value
        normalized_temp = (temperature - min_temp) / (max_temp - min_temp)
        green = int(255 * (1 - normalized_temp))
        red = int(255 * normalized_temp)
        color = (red, green, 0)  # Set blue channel to 0

    # Fill the corresponding pixel in the image array with the interpolated color
    image_array[y, x] = color

# Create and save the image
image = Image.fromarray(image_array)
image.save('temp_color_map_restored.bmp')

print("Image restored from temperature data and saved as temp_color_map_restored.jpg")


10 50
Image restored from temperature data and saved as temp_color_map_restored.jpg
